In [ ]:
%load_ext autoreload
%autoreload 2


import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from model import NeuralNetwork

df = pd.read_csv("data_v2.csv", sep=',')
df = df.query('n == 1')
df = df[["Method", "Beam Type", "L/h", "k", "result"]]
df = df.drop_duplicates(subset=["Method", "Beam Type", "L/h", "k"])

# shuffling the dataframe
df = df.sample(frac=1)

# preprocessing the data
method_dict = {
    "CBT":0,
    "TBT":1,
    "PSDBT":2,
    "ESDBT":3,
    "HSBDT":4,
    "HSDBT":4,
    "TSDBT":5,
    "ASDBT":6,
    "PESDBT":7,
    "ISDBT":8,
    "ICDBT":9,
    "ITDBT":10,
}

beam_type_dict = {
    "S-S FG":0,
    "C-F FG":1,
    "C-C FG":2
}

l_h_dict = {
    5:0,
    20:1,
    10:2,
    100:3,
    30:4
}


# l_h_mean, l_h_std = df["L/h"].mean(), df["L/h"].std()
# k_mean, k_std = df["k"].mean(), df["k"].std()
# df["L/h"] = (df["L/h"] - l_h_mean) / l_h_std
# df["k"] = (df["k"] - k_mean) / k_std
df["Method"] = df["Method"].map(method_dict)
df["Beam Type"] = df["Beam Type"].map(beam_type_dict)
df["L/h"] = df["L/h"].map(l_h_dict)


# to tensor
input_data = df[["Method", "Beam Type", "L/h", "k"]].to_numpy()
target_data = df["result"].to_numpy()

input_tensor = torch.tensor(input_data)

In [ ]:
method_one_hot = torch.nn.functional.one_hot(input_tensor[:,0].type(torch.int64), num_classes=len(method_dict.keys()))
beam_type_one_hot = torch.nn.functional.one_hot(input_tensor[:,1].type(torch.int64), num_classes=len(beam_type_dict.keys()))
l_h_one_hot = torch.nn.functional.one_hot(input_tensor[:,2].type(torch.int64), num_classes=len(l_h_dict.keys()))

X = torch.cat((method_one_hot, beam_type_one_hot, l_h_one_hot, input_tensor[:,3].unsqueeze(1)), dim=1).type(torch.float32)
y = torch.tensor(target_data).unsqueeze(1).type(torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_test, y_train, y_test = X_train.float(), X_test.float(), y_train.float(), y_test.float()

train_k_mean, train_k_std = torch.mean(X_train[:, -1]), torch.std(X_train[:, -1])
X_train[:, -1] = (X_train[:, -1] - train_k_mean) / train_k_std
X_test[:, -1] = (X_test[:, -1] - train_k_mean) / train_k_std

In [ ]:
# X = input_tensor.type(torch.float32)
# y = torch.tensor(target_data).unsqueeze(1).type(torch.float32)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# X_train, X_test, y_train, y_test = X_train.float(), X_test.float(), y_train.float(), y_test.float()

# train_k_mean, train_k_std = torch.mean(X_train[:, -1]), torch.std(X_train[:, -1])
# X_train[:, -1] = (X_train[:, -1] - train_k_mean) / train_k_std
# X_test[:, -1] = (X_test[:, -1] - train_k_mean) / train_k_std


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from skorch import NeuralNetRegressor
from skorch import helper

import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import parallel_backend

# with parallel_backend('multiprocessing'):
model = NeuralNetRegressor(
    NeuralNetwork,
    criterion=nn.MSELoss,
    max_epochs=300,
    batch_size=128,
    verbose=False,
    optimizer=optim.SGD,
    train_split=helper.predefined_split(torch.utils.data.TensorDataset(X_test, y_test))
)

param_grid = {
    ##### for optimizer
    # 'optimizer__lr': [0.01],
    # 'optimizer__momentum': [0.9],

    ##### for module
    'module__hidden_sizes':[[20, 20], [25, 25], [30, 30], [20, 20, 20], [25, 25, 25], [30, 30, 30]],
    # 'module__activation': [nn.ReLU, nn.Tanh, nn.Sigmoid],
    # 'module__dropout_rate': [0.1]
}

# Create your model, criterion, and grid search object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the grid search to find the best configuration
grid_result = grid_search.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

# Best: 0.440983 using {'module__activation': <class 'torch.nn.modules.activation.Sigmoid'>, 'module__dropout_rate': 0.1, 'module__hidden_size': 30, 'module__hidden_size2': 25, 'optimizer__lr': 0.01, 'optimizer__momentum': 0.9}
# Best: 0.498547 using {'module__activation': <class 'torch.nn.modules.activation.Sigmoid'>, 'module__dropout_rate': 0.1, 'module__hidden_size': 40, 'module__weight_constraint': 3.0, 'optimizer__lr': 0.01, 'optimizer__momentum': 0.9}
# Best: 0.622935 using {'module__activation': <class 'torch.nn.modules.activation.Tanh'>, 'module__dropout_rate': 0.1, 'module__hidden_size': 20, 'module__weight_constraint': 3.0, 'optimizer__lr': 0.01, 'optimizer__momentum': 0.9}

Best: 0.953049 using {'module__hidden_sizes': [30, 30]}
-0.183553 (1.592180) with: {'module__hidden_sizes': [20, 20]}
0.952865 (0.021380) with: {'module__hidden_sizes': [25, 25]}
0.953049 (0.020230) with: {'module__hidden_sizes': [30, 30]}
-0.321733 (1.789243) with: {'module__hidden_sizes': [20, 20, 20]}
0.951387 (0.023950) with: {'module__hidden_sizes': [25, 25, 25]}
-1.395386 (1.676329) with: {'module__hidden_sizes': [30, 30, 30]}


In [17]:
def test(model, test_dataloader):
    total_samples = 0
    total_mse = 0.0  # Initialize MSE to 0

    model.eval()
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            batch_size = labels.size(0)  # Get the batch size

            # Calculate the Mean Squared Error (MSE) for this batch
            mse = ((outputs - labels).pow(2).sum()).item()
            total_mse += mse
            total_samples += batch_size

    # Calculate the average MSE over the entire dataset
    mean_mse = total_mse / total_samples

    return mean_mse


# Create a TensorDataset
training_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


model = NeuralNetwork(hidden_sizes=[25, 25], activation=nn.ReLU, dropout_rate=0.1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# optimizer = optim.AdamW(model.parameters(), lr=0.01)

for epoch in range(300):
    cur_loss = 0
    total_len = 0
    for data, target in training_dataloader:  # Iterate through your data
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.float(), target.float())
        loss.backward()
        optimizer.step()

        cur_loss += loss
        total_len += data.shape[0]

    if epoch % 10 == 0:
        test_mse = test(model, test_dataloader)
        print(f"Epoch: {epoch} - Loss {cur_loss/total_len} - Test Mean MSE: {test_mse}")

Epoch: 0 - Loss 0.4828733205795288 - Test Mean MSE: 11.986270067168444
Epoch: 10 - Loss 0.019470922648906708 - Test Mean MSE: 0.9425389359637004
Epoch: 20 - Loss 0.005562742240726948 - Test Mean MSE: 0.31747904056456033
Epoch: 30 - Loss 0.004124201834201813 - Test Mean MSE: 0.18487190618747618
Epoch: 40 - Loss 0.0034989570267498493 - Test Mean MSE: 0.1488740967541206
Epoch: 50 - Loss 0.0034203112591058016 - Test Mean MSE: 0.11997310126700052
Epoch: 60 - Loss 0.0025313568767160177 - Test Mean MSE: 0.1081151148168052
Epoch: 70 - Loss 0.0029837931506335735 - Test Mean MSE: 0.26275413792307784
Epoch: 80 - Loss 0.0025142852682620287 - Test Mean MSE: 0.13302999589501358
Epoch: 90 - Loss 0.002039284911006689 - Test Mean MSE: 0.1391164500538896
Epoch: 100 - Loss 0.0019323122687637806 - Test Mean MSE: 0.08021355838310427
Epoch: 110 - Loss 0.001562196179293096 - Test Mean MSE: 0.06336010374673982
Epoch: 120 - Loss 0.0013727023033425212 - Test Mean MSE: 0.10829485916509861
Epoch: 130 - Loss 0.001

In [15]:
model.eval()
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        print(labels[4].item(), outputs[4].item())

1.3973000049591064 1.4225157499313354


In [16]:
method_one_hot = torch.nn.functional.one_hot(torch.tensor(4), num_classes=len(method_dict.keys()))
beam_type_one_hot = torch.nn.functional.one_hot(torch.tensor(0), num_classes=len(beam_type_dict.keys()))
l_h_one_hot = torch.nn.functional.one_hot(torch.tensor(1), num_classes=len(l_h_dict.keys()))
k = (0.5 - train_k_mean) / train_k_std

test_tensor = torch.cat((method_one_hot, beam_type_one_hot, l_h_one_hot, torch.tensor([k])), dim=0).type(torch.float32)

output = model(test_tensor)
output



tensor([4.7491], grad_fn=<ReluBackward0>)